# MODEL UPLOAD TO MONGODB DATA BASE

## 1 - Import libraries

It's necessary to load the model in MongoDB to create a docker image of all the project.

The idea is to be able to update the project with more pictures and models with more categories.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from pymongo import MongoClient
import pickle
import time

### 2 - Convert the model to binary data

In [ ]:
five_styles=pickle.dumps("./model/22_08_08-BN_pictures.h5")
print(cnn_model)

In [ ]:
first_try=keras.models.load_model("./model/22_08_08-BN_pictures.h5")
first_try

### 3 - Mongo connection and upload model

In [ ]:
##mongo_url= ADD HERE YOUR MONGO URL 

client = MongoClient(mongo_url)
db = client.get_database("Data")
model_db= db.Cnn_model

In [ ]:
mongo_model=model_db.insert_one({'model':five_styles, "name":"5 styles", "creation":time.time()})    


### 4 - Validate process and load model

In [ ]:
##mongo_url= ADD HERE YOUR MONGO URL 

client = MongoClient(mongo_url)
db = client.get_database("Data")
model_db= db.Cnn_model

In [ ]:
from bson import json_util
from json import loads
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder
from PIL import Image

In [ ]:
res=list(db["Cnn_model"].find())

solution=res[0]['creation']
print(solution)

for i in range(len(res)):
    if res[i]['creation']>solution:
        solution=res[i]['creation']

print(solution)

new_model=db["Cnn_model"].find({'creation':solution})

print(new_model)

for i in new_model:
    json_data=i
    print(i)

pickled_model=json_data['model']

new_mod=pickle.loads(pickled_model)
print(new_mod)



In [ ]:
classes=["Postmodern achitecture","Novelty architecture","International style", "Deconstructivsm", "Chicago school architecture"]

X_aux=[]
#creating classes
enc=OneHotEncoder(handle_unknown='ignore')

y=np.array(classes)
y=y.reshape(-1,1)
y_ohe=enc.fit_transform(y).toarray()

# create an array with the right shape
img=Image.open("./examples/chicago school/Auditorium_Building.jpg").resize((200,200)).convert("L")
img.show()
g_img=np.expand_dims(img, axis=-1)
g_img_chan=g_img.repeat(3, axis=-1)
X_aux.append(g_img_chan)
X_img=np.array(X_aux)
print(f"g_img_chan.shape: {g_img_chan.shape}")
print(f"X_img.shape: {X_img.shape}")

model=keras.models.load_model(new_mod)
pred=model.predict(X_img)
pred_text = enc.inverse_transform(pred)

results=[pred_text, pred]

In [ ]:
results